# GMFs and LUTs

In [ ]:
import xsar
from xsarsea import windspeed
import xarray as xr
import numpy as np
import holoviews as hv
hv.extension('bokeh')

In [ ]:
@windspeed.gmfs.register_gmf(inc_range=[17., 50.], wspd_range=[3., 80.], pols=['VH'], units='linear')
def gmf_dummy(incidence, speed, phi=None): 
    a0 = 0.00013106836021008122
    a1 = -4.530598283705591e-06
    a2 = 4.429277425062766e-08
    b0 = 1.3925444179360706
    b1 = 0.004157838450541205
    b2 = 3.4735809771069953e-05

    a = a0 + a1 * incidence + a2 * incidence ** 2
    b = b0 + b1 * incidence + b2 * incidence ** 2
    sig = a * speed ** b

    return sig

In [ ]:
sarwing_luts_subset_path = xsar.get_test_file('sarwing_luts_subset')
windspeed.sarwing_luts.register_all_sarwing_luts(sarwing_luts_subset_path)
windspeed.available_models

In [ ]:

def model_compare(compare_models):
    
    luts = [ windspeed.available_models[name].to_lut(units='linear')  for name in compare_models]
    
    if 'phi' not in luts[0].dims:
        kdims=['incidence']
        dim_range=dict(incidence=(17,50))
    else:
        kdims=['incidence', 'phi']
        dim_range=dict(incidence=(17,50), phi=(0,180))
    
    def model_curve(incidence, phi=None):
        if 'phi' not in luts[0].dims:
            sel = dict(incidence=incidence)
        else:
            sel = dict(phi=phi, incidence=incidence)
        return hv.Overlay(
            [
                hv.Curve(lut.sel(**sel, method='nearest'),'wspd','sigma0', label=lut.attrs['model']) for lut in luts
            ]
        )
    
    
    dmap = hv.DynamicMap(model_curve, kdims=kdims).opts(height=600, width=600)
    return dmap.redim.range(**dim_range)
    


In [ ]:
model_compare([ 'gmf_dummy', 'sarwing_lut_cmodms1ahw' ])

In [ ]:
model_compare([ 'gmf_cmod5n', 'gmf_cmod5' ])